In [1]:
import os
#модуль для работы с ftp
import ftplib
#модуль для работы с zip архивами
from zipfile import *
#Библиотека для работы с Oracle
import cx_Oracle
import pandas as pd
import datetime as dt
import time

In [2]:
#!!!Важно. Установленная по умолчанию в Canopy версия библиотеки lxml не умеет читать кодировку windows-1251
#Поэтому необходимо скачать с сайта https://pypi.python.org/pypi/lxml последнюю версию (3.7.3) библиотеки 
#Подключаем библиотеку для парсинга XML
import lxml.etree as et

In [3]:
# Для корректного отображения оракловых сообщений
os.environ['NLS_LANG'] = 'AMERICAN_AMERICA.AL32UTF8'

In [4]:
gloglist=[]
def log(atext):
    global gloglist
    gloglist.append(time.asctime(time.localtime(time.time()))+' | '+atext+'\n')

In [5]:
def savelog():
    global gloglist
    logfile=open('z:\\Python\\zakupki\\log.txt','w') #Создаем новый файл в режиме записи
    try:
        logfile.writelines(gloglist)
    finally:
        logfile.close()
    gloglist=[]    

In [6]:
#Процедура скачивает файл filename с указанной директории directory ftp и кладет его по указанному локальному пути 
def download(ftp,directory,loc_filepath,filename):
    ftp.cwd(directory) #Задает текущую директорию на ftp
    loc_file_name=os.path.join(loc_filepath, filename) #конкатенирует имя файла с путем
    #Открывает файл для записи в двоичном формате. Указатель стоит в начале файла. 
    #Создает файл с именем имя_файла, если такового не существует.
    f = open(loc_file_name,"wb")
    #Получить файл в двоичном режиме. Команда должна иметь соответствующую команду RETR: 'RETR имя_файла'.
    ftp.retrbinary("RETR " + filename,f.write)
    f.close()

In [7]:
#Функция возвращает список файлов или папок в указанной директории на ftp
def get_ftp_dir_list(ftp, ftp_path):
    ftp.cwd(ftp_path) #Задает текущую директорию на ftp
    llist = []
    #Получение файла или каталога в ASCII режиме передачи. 
    #Команда должна иметь соответствующую команду RETR (см. retrbinary ()) или команду такую как LIST, NLST или MLSD
    ftp.retrlines("LIST", llist.append)
    #т.к. в полученном списке присутствуют все атрибуты файла или папки, надо вычленить только имя
    llist=[s.split(None, 8)[-1].lstrip() for s in llist]
    return llist

In [8]:
def unzipall(path,archname):
    #Проверяем, является ли указанный файл zip архивом
    if is_zipfile(path + archname):
        #Открываем архив на чтение
        z = ZipFile(path + archname, 'r')
        #Распаковываем все файлы из архива по указанному пути
        z.extractall(path)
        #Возвращаем список файлов в архиве
        return z.namelist()
        z.close()
    else:
        print('Некорректный архив : ' +  archname)

In [9]:
def str_to_date(astrdate):
    #Даты в xml пишутся по разному. 2015-01-16T00:00:00 или 2015-01-16
    if len(astrdate)==0:
        return None
    elif astrdate[0].text[0]=='0':
        return None
    elif ('T' in astrdate[0].text):
        return dt.datetime.strptime(astrdate[0].text, '%Y-%m-%dT%H:%M:%S')
    else:
        return dt.datetime.strptime(astrdate[0].text, '%Y-%m-%d')

In [10]:
def xpath_nulls(atag_value):
    if len(atag_value)==0:
        return None
    else:
        return atag_value[0].text

In [11]:
def xpath_float(atag_value):
    if len(atag_value)==0:
        return 0
    else:
        return float(atag_value[0].text)

In [12]:
#Функция парсинга xml
def parse_xml(afile, adoc):
    #парсим файл XML и создаем дерево елементов element tree
    etxml = et.parse(afile)
    #получаем корневой элемент дерева
    root=etxml.getroot()
    #Данный цикл позволяет получить список всех элементов (тэгов) дерева, названий и значений
    #Это особенно актуально, когда перед элементом указывается пространство имен (name space), например
    #{http://zakupki.gov.ru/223fz/contract/1}contractRegNumber
    #Это нужно при поиске конкретных тэгов с помощью функций xpath и findall
    #for item in root.iterfind('.//'):
    #    print item.tag + '  :  ' + item.text
    #В аргументе функции xpath указываем строку для поиска, здесь './/' означает, 
    #что надо искать не только среди потомков корневого элемента, но во всех элементах дерева вообще
    #также перечисляем все пространства имен, используемые перед названием тэга, их можно взять из заголовка xml
    #они находятся после ключа xmlns= (для тэгов без указания пространства имен, например <inn>5260267654</inn>) 
    #или xmlns: (для тэгов с указанием пространтства имен, например <ns2:code>40000</ns2:code> ), 
    #сами название пространств имен в xpath не обязательно должны совпадать с xml, но пути и URL должны совпадать
    if adoc=='protocolvk':
        lnamespaces={'ns2' : 'http://zakupki.gov.ru/223fz/purchase/1', 'ns':'http://zakupki.gov.ru/223fz/types/1'}
        acontract=[ 
            root.xpath('.//ns2:purchaseInfo/ns:purchaseNoticeNumber',namespaces=lnamespaces)[0].text,
            root.xpath('.//ns2:purchaseInfo/ns:purchaseMethodCode',namespaces=lnamespaces)[0].text,
            root.xpath('.//ns2:purchaseInfo/ns:purchaseCodeName',namespaces=lnamespaces)[0].text,
            [s.text for s in root.xpath('.//ns2:lotApplicationsList/ns2:protocolLotApplications/ns2:application/ns2:supplierInfo/ns:name',namespaces=lnamespaces)],
            ]
    #Альтернативная xpath функция, но с меньшими возможностями и немного отличным синтаксисом поискового запроса
    #lm = root.findall('.//{http://zakupki.gov.ru/223fz/contract/1}contractRegNumber')
    return acontract

In [13]:
def compdate(afname,adfrom,adto):
    ifrom=afname.find('_000000_')
    if ifrom==-1:
        return False
    else:
        ldfile=afname[ifrom+8:ifrom+16]
        return dt.datetime.strptime(adfrom, '%Y%m%d')\
               <=dt.datetime.strptime(ldfile, '%Y%m%d')\
               <=dt.datetime.strptime(adto, '%Y%m%d')

In [14]:
def complists(alist1, alist2):
    for i in alist1:
        for j in alist2:
            si=unicode(i)+'.'
            sj=unicode(j)+'.'
            if si.find(sj)==0:
                return True
    return False

In [15]:
#Загружаем все файлы с ftp госзакупок из подпапки ftp_dir  в локальную папку lxmlpath
#def gz_get_ftp_files(ftp, ftp_dir, lxmlpath, adf_orgs, adoc, adfrom, adto, aregions=None):
def gz_get_ftp_files(ftp, ftp_dir, lxmlpath, anotices, adoc, adfrom, adto, aregions=None):
    #создаем DataFrame для данных контрактов
    contracts=pd.DataFrame(columns=['nnumber','mcode','mname','nsupplier','ncount',
                                    'zip','xml','region'])
    for s in get_ftp_dir_list(ftp,"/out/published/"):
        i=0
        #s=get_ftp_dir_list(ftp, "/out/published/")[0]
        if aregions==None or ((aregions!=None) and (s in aregions)):
            try:
                lpath="/out/published/" + s + ftp_dir
                time.asctime(time.localtime(time.time()))
                print(time.asctime(time.localtime(time.time())) + ' | Каталог: ' + s)
                ftp.cwd(lpath)
                lftplist=get_ftp_dir_list(ftp, lpath)
                print('Всего файлов (архивов) в FTP каталоге: ' +str(len(lftplist)))
                for f in lftplist:
                    if (f[-4:]=='.zip') and compdate(f,adfrom,adto):
                        log(f)
                        download(ftp, lpath,lxmlpath,f)
                        i+=1
                        archlist=unzipall(lxmlpath,f)
                        #Поскольку в разных архивах содержатся XML файлы с одинаковыми названиями 
                        #- добавляем к названию файла уникальный для архива номер, либо после парсинга сразу удаляем файл XML
                        for x in archlist:
                            #если файл не нулевой длины - парсим xml и загружаем данные в DataFrame
                            if os.path.getsize(lxmlpath + x)!= 0:
                                log(x)
                                contract=parse_xml(lxmlpath + x, adoc)
                                # Удаляем дубли в списке оквэд
                                if contract[0] in anotices:
                                    #добавляем имена файлов zip и xml в список
                                    contract.append(len(contract[3]))
                                    contract.append(f)
                                    contract.append(x)
                                    contract.append(s)
                                    #Могут досылать исправленные данные по контракту в разные дни. Это надо обрабатывать.
                                    #!!!Данный функционал работает не стабильно!!!
                                    #Если мы присваиваем один DataFrame другому или один List другому с помощью оператора =, 
                                    #то в этом случае объект остается один и тот же. Копируется только ссылка на него
                                    #Функция copy() и list() позволяет скопировать физически сам объект и избежать сообщения:
                                    #A value is trying to be set on a copy of a slice from a DataFrame
                                    #tmpdf=contracts[(contracts['number']==contract[0])&(contracts['date']==contract[1])].copy()
                                    #if len(tmpdf.index)!=0:
                                    #    tmpdf.loc[0]=list(contract)
                                    #    contracts.loc[contracts[(contracts['number']==contract[0])&(contracts['date']==contract[1])].index,tmpdf.columns]=tmpdf.copy()
                                    #    print str(contracts[(contracts['number']==contract[0])&(contracts['date']==contract[1])].index)+' upd '.join(list(map(unicode,contract)))
                                    #else:
                                    contracts.loc[len(contracts)]=contract
                                    #    print ' add '.join(list(map(unicode,contract)))
                                #os.rename(lxmlpath + x,lxmlpath + x[:-4] + '_' + str(i) + x[-4:])
                            os.remove(lxmlpath + x)    
                        #удаляем файл архива    
                        os.remove(lxmlpath + f)
                        if i%100==0:
                            print(time.asctime(time.localtime(time.time())) 
                            + ' | Обработано '+str(i)+' файлов из '+str(len(lftplist)) 
                            +' ('+str(int(round(float(i)/len(lftplist)*100)))+'%)')
            except ftplib.error_perm:
                print('В папке ' + s + ' каталог ' + ftp_dir + ' не существует!')
    #Могут досылать исправленные данные по контракту в разные дни. Это надо обрабатывать.
    contracts=contracts.drop_duplicates(['nnumber'],keep='last')
    contracts.index=range(len(contracts.index))
    print(u'Окончание обработки: '+time.asctime(time.localtime(time.time())))
    savelog()
    return contracts

In [16]:
#Блок подготовки и проверки SQL команды
def insert_prepare(asql,acursor):
    try:
        acursor.prepare(asql)
    except cx_Oracle.DatabaseError as exception:
        print('Failed to prepare cursor')
        print(exception)
        exit(1)

In [17]:
def load_from_ora(asql):    
    conn=cx_Oracle.connect('analytics/analytics@fst_rac')
    cursor=conn.cursor()
    try:
        insert_prepare(asql,cursor)
        try:
            cursor.execute(cursor.statement)
            ltable=cursor.fetchall()
            #получаем названия полей запроса
            lcols=[col_desc[0] for col_desc in cursor.description]
        except cx_Oracle.DatabaseError as exception:
            print('Failed to insert row')
            print(exception)
            print(cursor.statement)
            exit(1)
        df=pd.DataFrame(ltable, columns=lcols)
        return df
    finally:
        cursor.close()
        conn.close()

In [18]:
def load_df_to_ora(adf):
    conn=cx_Oracle.connect('analytics/analytics@fst_rac')
    cursor=conn.cursor()
    try:
        isql = """
                  INSERT INTO ZAKUPKI_PROTOKOLVK 
                  (nnumber, mcode, mname, nsupplier, ncount, zip, nxml, nregion) 
                  VALUES (:s_nnumber, :s_mcode, :s_mname, :s_nsupplier, :s_ncount, :s_zip, :s_nxml, :s_nregion)
               """
        #Блок подготовки и проверки SQL команды
        for i in range(len(adf.index)):
            #if adf['NOTICENUM'][i]=='NULL':
            insert_prepare(isql,cursor)
            #else:
            #    insert_prepare(usql,cursor)
            try:
                #Выполнение Insert
                cursor.execute (cursor.statement,s_nnumber=adf['nnumber'][i],
                                s_mcode=adf['mcode'][i],s_mname=adf['mname'][i],
                                s_nsupplier=None,s_ncount=adf['ncount'][i],
                                s_zip=adf['zip'][i],s_nxml=adf['xml'][i], s_nregion=adf['region'][i]
                                )
            except cx_Oracle.DatabaseError as exception:
                print('Failed to insert row')
                print(exception)
                print(cursor.statement)
                #print str(df['NYEAR'][i])+'|'+str(df['GROUP_NUMBER'][i])+'|'+str(df['GROUP_NAME'][i])+'|'+str(df['ORG_REG_NUMBER'][i])+'|'+str(df['ORG_NAME'][i])
                exit(1)
            if (i>0) and (i%100==0):
                print(u' Обработано строк: '+str(i))
        conn.commit()   
    finally:
        cursor.close()
        conn.close()

In [19]:
sql=u"""
    select noticenum
    from analytics.ZAKUPKI_SMP
    """      
df_notices=load_from_ora(sql)

In [20]:
def ftpconnect():
    #создаем объект ftp для заданного адреса ftp-сервера
    lftp = ftplib.FTP("ftp.zakupki.gov.ru")
    #коннектимся к серверу с именем пользователя и паролем
    lftp.login("fz223free", "fz223free")
    return lftp

In [21]:
ftp=ftpconnect()
get_ftp_dir_list(ftp,'/out/published/')

['Adygeya_Resp',
 'Altay_Resp',
 'Altayskii__krai',
 'Amurskaya_obl',
 'Arhangelskaya_obl',
 'Astrahanskaya_obl',
 'Baikonur_g',
 'Bashkortostan_Resp',
 'Belgorodskaya_obl',
 'Brianskaya_obl',
 'Buryatiya_Resp',
 'Chechenskaya_Resp',
 'Cheliabinskaya_obl',
 'Chukotskii_AO',
 'Chuvashskaya_Respublika',
 'Dagestan_Resp',
 'Evreiskaya_Aobl',
 'Habarovskii_krai',
 'Hakasiia_Resp',
 'Hanty-Mansiiskii_AO_Iugra_AO',
 'Ingushetiya_Resp',
 'Irkutskaya_obl',
 'Irkutskaya_obl_Ust-Ordynskii_Buriatskii_okrug',
 'Ivanowskaya_obl',
 'Jamalo-Nenetckii_AO',
 'Jaroslavskaya_obl',
 'Kabardino-Balkarskaya_Resp',
 'Kaliningradskaya_obl',
 'Kalmykiya_Resp',
 'Kaluzhskaya_obl',
 'Kamchatskii_krai',
 'Karachaevo-Cherkesskaya_Resp',
 'Kareliya_Resp',
 'Kemerowskaya_obl',
 'Kirowskaya_obl',
 'Komi_Resp',
 'Kostromskaya_obl',
 'Krasnodarskii_krai',
 'Krasnoyarskii_krai',
 'Krym_Resp',
 'Kurganskaya_obl',
 'Kurskaya_obl',
 'Leningradskaya_obl',
 'Lipetckaya_obl',
 'Magadanskaya_obl',
 'Marii_El_Resp',
 'Mordoviya

In [22]:
lreglist=[
 'Adygeya_Resp',
 'Altay_Resp',
 'Altayskii__krai',
 'Amurskaya_obl',
 'Arhangelskaya_obl',
 'Astrahanskaya_obl',
 'Baikonur_g',
 'Bashkortostan_Resp',
 'Belgorodskaya_obl',
 'Brianskaya_obl',
 'Buryatiya_Resp',
 'Chechenskaya_Resp',
 'Cheliabinskaya_obl',
 'Chukotskii_AO',
 'Chuvashskaya_Respublika',
 'Dagestan_Resp',
 'Evreiskaya_Aobl',
 'Habarovskii_krai',
 'Hakasiia_Resp',
 'Hanty-Mansiiskii_AO_Iugra_AO',
 'Ingushetiya_Resp',
 'Irkutskaya_obl',
 'Irkutskaya_obl_Ust-Ordynskii_Buriatskii_okrug',
 'Ivanowskaya_obl',
 'Jamalo-Nenetckii_AO',
 'Jaroslavskaya_obl',
 'Kabardino-Balkarskaya_Resp',
 'Kaliningradskaya_obl',
 'Kalmykiya_Resp',
 'Kaluzhskaya_obl',
 'Kamchatskii_krai',
 'Karachaevo-Cherkesskaya_Resp',
 'Kareliya_Resp',
 'Kemerowskaya_obl',
 'Kirowskaya_obl',
 'Komi_Resp',
 'Kostromskaya_obl',
 'Krasnodarskii_krai',
 'Krasnoyarskii_krai',
 'Krym_Resp',
 'Kurganskaya_obl',
 'Kurskaya_obl',
 'Leningradskaya_obl',
 'Lipetckaya_obl',
 'Magadanskaya_obl',
 'Marii_El_Resp',
 'Mordoviya_Resp',
 'Moskovskaya_obl',
 'Moskva',
 'Murmanskaya_obl',
 'Nenetckii_AO',
 'Nizhegorodskaya_obl',
 'Novgorodskaya_obl',
 'Novosibirskaya_obl',
 'Omskaya_obl',
 'Orenburgskaya_obl',
 'Orlovskaya_obl',
 'Penzenskaya_obl',
 'Permskii_krai',
 'Primorskii_krai',
 'Pskovskaya_obl',
 'Rostovskaya_obl',
 'Ryazanskaya_obl',
 'Saha_Jakutiya_Resp',
 'Sahalinskaya_obl',
 'Samarskaya_obl',
 'Sankt-Peterburg',
 'Saratovskaya_obl',
 'Sevastopol',
 'Severnaia_Osetiya_Alaniia_Resp',
 'Smolenskaya_obl',
 'Stavropolskii_krai',
 'Sverdlovskaya_obl',
 'Tambovskaya_obl',
 'Tatarstan_Resp',
 'Tiumenskaya_obl',
 'Tomskaya_obl',
 'Tulskaya_obl',
 'Tverskaya_obl',
 'Tyva_Resp',
 'Udmurtskaya_Resp',
 'Ulianovskaya_obl',
 'Vladimirskaya_obl',
 'Volgogradskaya_obl',
 'Vologodskaya_obl',
 'Voronezhskaya_obl',
 'Zabaikalskii_krai',
 'Zabaikalskii_krai_Aginskii_Buriatskii_okrug'
]

In [24]:
lreglist=[
 'Permskii_krai',
 'Primorskii_krai',
 'Pskovskaya_obl',
 'Rostovskaya_obl',
 'Ryazanskaya_obl',
 'Saha_Jakutiya_Resp',
 'Sahalinskaya_obl',
 'Samarskaya_obl',
 'Sankt-Peterburg',
 'Saratovskaya_obl',
 'Sevastopol',
 'Severnaia_Osetiya_Alaniia_Resp',
 'Smolenskaya_obl',
 'Stavropolskii_krai',
 'Sverdlovskaya_obl',
 'Tambovskaya_obl',
 'Tatarstan_Resp',
 'Tiumenskaya_obl',
 'Tomskaya_obl',
 'Tulskaya_obl',
 'Tverskaya_obl',
 'Tyva_Resp',
 'Udmurtskaya_Resp',
 'Ulianovskaya_obl',
 'Vladimirskaya_obl',
 'Volgogradskaya_obl',
 'Vologodskaya_obl',
 'Voronezhskaya_obl',
 'Zabaikalskii_krai',
 'Zabaikalskii_krai_Aginskii_Buriatskii_okrug'
]

In [26]:
# purchaseProtocolRZOK - протоколы открытого конкурса
# purchaseProtocolRZOA - протоколы открытого аукциона
# purchaseProtocolRZAE - протоколы аукциона в электронной форме
# purchaseProtocol
for reg in lreglist:
    ftp=ftpconnect()
    contracts=gz_get_ftp_files(ftp, "/purchaseProtocol/daily/", "z:\\Python\\zakupki\\contracts_xml\\",df_notices.NOTICENUM.tolist(),
                          'protocolvk','20180101','20180831',[reg])
    load_df_to_ora(contracts)


Tue Oct 09 12:57:44 2018 | Каталог: Permskii_krai
Всего файлов (архивов) в FTP каталоге: 2717
Tue Oct 09 12:58:25 2018 | Обработано 100 файлов из 2717 (4%)
Tue Oct 09 12:59:09 2018 | Обработано 200 файлов из 2717 (7%)
Tue Oct 09 12:59:45 2018 | Обработано 300 файлов из 2717 (11%)
Окончание обработки: Tue Oct 09 12:59:58 2018
 Обработано строк: 100
 Обработано строк: 200
 Обработано строк: 300
 Обработано строк: 400
 Обработано строк: 500
 Обработано строк: 600
 Обработано строк: 700
 Обработано строк: 800
 Обработано строк: 900
 Обработано строк: 1000
 Обработано строк: 1100
 Обработано строк: 1200
 Обработано строк: 1300
 Обработано строк: 1400
 Обработано строк: 1500
 Обработано строк: 1600
 Обработано строк: 1700
 Обработано строк: 1800
 Обработано строк: 1900
 Обработано строк: 2000
 Обработано строк: 2100
 Обработано строк: 2200
Tue Oct 09 13:00:08 2018 | Каталог: Primorskii_krai
Всего файлов (архивов) в FTP каталоге: 1802
Tue Oct 09 13:00:54 2018 | Обработано 100 файлов из 1802 (

In [83]:
pd.set_option('max_colwidth',150)
contracts

,nnumber,mcode,mname,nsupplier,ncount,zip,xml,region
0,31705975292,110918,Запрос предложений,"[ООО ""Новомариинский ТПК""]",1.0,purchaseProtocol_Chukotskii_AO_20180111_000000_20180111_235959_daily_001.xml.zip,purchaseProtocol_Chukotskii_AO_20180111_000000_20180111_235959_daily_001.xml,Chukotskii_AO
1,31705859229,924,открытый запрос предложений (ЭТП),[Публичное акционерное общество «Сбербанк России»],1.0,purchaseProtocol_Chukotskii_AO_20180112_000000_20180112_235959_daily_001.xml.zip,purchaseProtocol_Chukotskii_AO_20180112_000000_20180112_235959_daily_002.xml,Chukotskii_AO
2,31705912117,28891,запрос цен открытый (ЭТП),"[ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ФОРРУС"", Общество с ограниченной ответственностью ""Нефтехиммаш""]",2.0,purchaseProtocol_Chukotskii_AO_20180115_000000_20180115_235959_daily_001.xml.zip,purchaseProtocol_Chukotskii_AO_20180115_000000_20180115_235959_daily_003.xml,Chukotskii_AO
3,31705904843,924,открытый запрос предложений (ЭТП),[Общество с ограниченной ответственностью «Альбатрос Т»],1.0,purchaseProtocol_Chukotskii_AO_20180118_000000_20180118_235959_daily_001.xml.zip,purchaseProtocol_Chukotskii_AO_20180118_000000_20180118_235959_daily_002.xml,Chukotskii_AO
4,31806122511,2068,запрос цен открытый (ЭТП),"[Общество с ограниченной ответственностью ""ТехноГарант"", Общество с ограниченной ответственностью «Доринжиниринг»]",2.0,purchaseProtocol_Chukotskii_AO_20180305_000000_20180305_235959_daily_001.xml.zip,purchaseProtocol_Chukotskii_AO_20180305_000000_20180305_235959_daily_002.xml,Chukotskii_AO
5,31806163022,28873,открытый запрос предложений (ЭТП),"[Общество с ограниченной ответственностью «НПП Бреслер"", ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""Промис"", Общество с Ограниченной Ответственнос...",4.0,purchaseProtocol_Chukotskii_AO_20180316_000000_20180316_235959_daily_001.xml.zip,purchaseProtocol_Chukotskii_AO_20180316_000000_20180316_235959_daily_005.xml,Chukotskii_AO
6,31806163059,2068,запрос цен открытый (ЭТП),"[Акционерное общество ""Холдинговая компания ""Энергомаш-Строй"", Акционерное общество ""Белэнергомашсервис"", Общество с ограниченной ответственностью...",6.0,purchaseProtocol_Chukotskii_AO_20180320_000000_20180320_235959_daily_001.xml.zip,purchaseProtocol_Chukotskii_AO_20180320_000000_20180320_235959_daily_001.xml,Chukotskii_AO
7,31806234213,41086,Запрос предложений,"[ООО ""Новомариинский ТПК""]",1.0,purchaseProtocol_Chukotskii_AO_20180321_000000_20180321_235959_daily_001.xml.zip,purchaseProtocol_Chukotskii_AO_20180321_000000_20180321_235959_daily_001.xml,Chukotskii_AO
8,31806175178,924,открытый запрос предложений (ЭТП),"[Общество с ограниченной ответственностью ""Гермес"", Общество с ограниченной ответственностью ""Завод электротехнического оборудования""]",2.0,purchaseProtocol_Chukotskii_AO_20180321_000000_20180321_235959_daily_001.xml.zip,purchaseProtocol_Chukotskii_AO_20180321_000000_20180321_235959_daily_003.xml,Chukotskii_AO
9,31705232331,152899,запрос предложений,[],0.0,purchaseProtocol_Chukotskii_AO_20180322_000000_20180322_235959_daily_001.xml.zip,purchaseProtocol_Chukotskii_AO_20180322_000000_20180322_235959_daily_001.xml,Chukotskii_AO


### Создать таблицу 
create table ZAKUPKI_PROTOKOLVK
(
  nnumber   VARCHAR2(100),
  mcode     VARCHAR2(4),
  mname     VARCHAR2(1000),
  nsupplier VARCHAR2(1000),
  ncount    number(4),
  zip       VARCHAR2(255),
  nxml      VARCHAR2(255),
  nregion   VARCHAR2(255)
)
tablespace USERS
  pctfree 10
  initrans 1
  maxtrans 255
  storage
  (
    initial 64K
    next 1M
    minextents 1
    maxextents unlimited
  );

In [146]:
contracts['okpd2']=contracts['okpd2'].apply('; '.join)
contracts['okpd']=contracts['okpd'].apply('; '.join)
contracts['okdp']=contracts['okdp'].apply('; '.join)

In [85]:
load_df_to_ora(contracts)

 Обработано строк: 100
 Обработано строк: 200
 Обработано строк: 300
 Обработано строк: 400
 Обработано строк: 500
 Обработано строк: 600
 Обработано строк: 700
 Обработано строк: 800
 Обработано строк: 900
 Обработано строк: 1000
 Обработано строк: 1100
 Обработано строк: 1200
 Обработано строк: 1300
 Обработано строк: 1400
 Обработано строк: 1500
 Обработано строк: 1600
 Обработано строк: 1700
 Обработано строк: 1800
 Обработано строк: 1900
 Обработано строк: 2000
 Обработано строк: 2100
 Обработано строк: 2200
 Обработано строк: 2300
 Обработано строк: 2400
 Обработано строк: 2500
 Обработано строк: 2600
 Обработано строк: 2700
 Обработано строк: 2800
 Обработано строк: 2900
 Обработано строк: 3000
 Обработано строк: 3100
 Обработано строк: 3200
 Обработано строк: 3300
 Обработано строк: 3400
 Обработано строк: 3500
 Обработано строк: 3600
 Обработано строк: 3700
 Обработано строк: 3800
 Обработано строк: 3900
 Обработано строк: 4000
 Обработано строк: 4100
 Обработано строк: 4200
 